In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
transform = transforms.Compose([transforms.Pad(4),transforms.RandomHorizontalFlip(),transforms.RandomCrop(32),transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=True,transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../../data/', train=False,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=100, shuffle=False)


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [0]:
class ResBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, compr=None):
    super(ResBlock, self).__init__()
    self.conv1=nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU(inplace=True)
    self.conv2=nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=1, padding=1, bias=False)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.compr=compr

  def forward(self,x):
    res=x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    if self.compr:
      res=self.compr(x)
    out = out+res 
    out= self.relu(out)
    return out
    

In [0]:
class Bottleneck(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, compr=None):
    super(Bottleneck, self).__init__()
    self.conv1=nn.Conv2d(in_channels, out_channels//4, kernel_size=1,stride=stride, padding=0, bias=False)
    self.bn1=nn.BatchNorm2d(out_channels//4)
    self.relu=nn.ReLU(inplace=True)  
    self.conv2=nn.Conv2d(out_channels//4, out_channels//4, kernel_size=3,stride=1, padding=1, bias=False)
    self.bn2=nn.BatchNorm2d(out_channels//4)
    self.conv3=nn.Conv2d(out_channels//4, out_channels, kernel_size=1,stride=1, padding=0, bias=False)
    self.bn3=nn.BatchNorm2d(out_channels)

    self.compr=compr
  def forward(self,x):
    res=x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)
    out=self.conv3(out)
    out=self.bn3(out)
    if self.compr:
      res=self.compr(x)
    out = out+res 
    out= self.relu(out)
    return out


In [0]:
class Net(nn.Module):
    def __init__(self, block, layers, num_classes=10):
       super(Net, self).__init__()
       self.in_channels=16
       self.conv=nn.Conv2d(3,16, kernel_size=3,stride=1, padding=1, bias=False)
       self.bn=nn.BatchNorm2d(16)
       self.relu = nn.ReLU(inplace=True)
       self.l1 = self.make_layer(block, 16, layers[0])
       self.l2 = self.make_layer(block, 32, layers[1], 2)
       self.l3 = self.make_layer(block, 64, layers[2], 2)

       self.avg_pool = nn.AvgPool2d(8)
       self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
       compr = None
       if (stride != 1) or (self.in_channels != out_channels):
            compr = nn.Sequential( nn.Conv2d(self.in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False), nn.BatchNorm2d(out_channels))
       layers = []
       layers.append(block(self.in_channels, out_channels, stride, compr))
       self.in_channels=out_channels
       for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
       return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.l1(out)
        out = self.l2(out)
        out = self.l3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
model1=Net(ResBlock,[10,10,10]).to(device)
learn_rate=.001
epochs=60

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=learn_rate)
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [8]:
total_step = len(train_loader)
curr_lr = learn_rate
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1,epochs, i+1, total_step, loss.item()))   
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)


Epoch [1/60], Step [100/500] Loss: 1.8782
Epoch [1/60], Step [200/500] Loss: 1.6295
Epoch [1/60], Step [300/500] Loss: 1.4962
Epoch [1/60], Step [400/500] Loss: 1.5189
Epoch [1/60], Step [500/500] Loss: 1.3959
Epoch [2/60], Step [100/500] Loss: 1.1980
Epoch [2/60], Step [200/500] Loss: 1.2169
Epoch [2/60], Step [300/500] Loss: 1.1631
Epoch [2/60], Step [400/500] Loss: 1.1814
Epoch [2/60], Step [500/500] Loss: 1.1159
Epoch [3/60], Step [100/500] Loss: 0.9979
Epoch [3/60], Step [200/500] Loss: 0.9639
Epoch [3/60], Step [300/500] Loss: 0.9357
Epoch [3/60], Step [400/500] Loss: 0.8260
Epoch [3/60], Step [500/500] Loss: 0.7338
Epoch [4/60], Step [100/500] Loss: 0.9812
Epoch [4/60], Step [200/500] Loss: 0.9041
Epoch [4/60], Step [300/500] Loss: 0.7997
Epoch [4/60], Step [400/500] Loss: 0.7135
Epoch [4/60], Step [500/500] Loss: 0.7018
Epoch [5/60], Step [100/500] Loss: 0.6008
Epoch [5/60], Step [200/500] Loss: 0.6558
Epoch [5/60], Step [300/500] Loss: 0.4736
Epoch [5/60], Step [400/500] Loss:

In [9]:
model1.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 89.7 %


In [0]:
model2=Net(Bottleneck,[5,5,5]).to(device)
learn_rate=.001
epochs=40

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=learn_rate)
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
total_step = len(train_loader)
curr_lr = learn_rate
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model2(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1,epochs, i+1, total_step, loss.item()))   
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/40], Step [100/500] Loss: 1.8777
Epoch [1/40], Step [200/500] Loss: 1.7330
Epoch [1/40], Step [300/500] Loss: 1.6805
Epoch [1/40], Step [400/500] Loss: 1.6939
Epoch [1/40], Step [500/500] Loss: 1.7696
Epoch [2/40], Step [100/500] Loss: 1.4729
Epoch [2/40], Step [200/500] Loss: 1.5062
Epoch [2/40], Step [300/500] Loss: 1.4576
Epoch [2/40], Step [400/500] Loss: 1.4239
Epoch [2/40], Step [500/500] Loss: 1.4624
Epoch [3/40], Step [100/500] Loss: 1.4365
Epoch [3/40], Step [200/500] Loss: 1.1488
Epoch [3/40], Step [300/500] Loss: 1.0587
Epoch [3/40], Step [400/500] Loss: 1.3796
Epoch [3/40], Step [500/500] Loss: 1.0651
Epoch [4/40], Step [100/500] Loss: 0.8905
Epoch [4/40], Step [200/500] Loss: 1.1196
Epoch [4/40], Step [300/500] Loss: 1.1045
Epoch [4/40], Step [400/500] Loss: 1.0105
Epoch [4/40], Step [500/500] Loss: 1.1764
Epoch [5/40], Step [100/500] Loss: 0.8956
Epoch [5/40], Step [200/500] Loss: 1.0195
Epoch [5/40], Step [300/500] Loss: 1.0061
Epoch [5/40], Step [400/500] Loss:

In [14]:
model2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 80.91 %
